FOR NMT STEPS WE NEED TO TAKE
- unicode to asqcui 
- preprocess sentences
- data exploration
- building vocabulary indexes
- pad sequnces
- dataset , dataloader class
- encoder building
- decorder building

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, batch_size):
        super(Encoder, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embed_layer = nn.Embedding(self.vocab_size, self.embed_size)
        self.gru = nn.GRU(self.embed_size, self.hidden_size)
    
    def forward(self, X):
        x = self.embedding(X) # batchSize, max_length, embed_dim
        ho = torch.zeros((1, self.batch_size, self.hidden_size))
        output, hidden = self.gru(x, ho)

        return output, hidden


In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, dec_hidden, enc_hidden, batch_size):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.dec_hidden = dec_hidden
        self.enc_hidden = enc_hidden
        self.batch_size = batch_size

        self.embed_layer = nn.Embedding(self.vocab_size, self.embed_size)
        self.gru = nn.GRU(self.embed_size + self.enc_hidden, self.dec_hidden, batch_first = True)
        self.fc = nn.Linear(self.dec_units, self.vocab_size)
    
    def forward(self, x, hidden, enc_output):
        enc_output = enc_output.premute(1, 0, 2)
        ho = torch.zeros((1, self.batch_size, self.dec_hidden))
        final_input = torch.cat((enc_output, hidden), dim = 1)
        output, state = self.gru(x, ho)

        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))

        x = self.fc(output)

        return x, state



